### Parsing one link and creating initial csv (ONE TIME USE)

This code Created the leading CSV file. It is intented to be ran first and only once. After running this code, move on to running Def 14A Parser Final

In [ ]:
# Creating Initial CSV (from single sample link)
sample = ["https://www.sec.gov/Archives/edgar/data/1001115/000119312519001874/d675272ddef14a.htm"]

# Import packages
import requests
import urllib
from bs4 import BeautifulSoup
import json
import pandas as pd
import re

tables = []
for link in sample:
    
    content = requests.get(link)
    soup = BeautifulSoup(content.text, 'lxml')

    try:
        search_result = pd.read_html(link, match="Beneficial", header=0)
        for df in search_result:
            df = df.astype('str')
            for column in df.columns:
                if df[column].str.lower().str.contains("%").any():
                    tables.append(df)
                    break
    except ValueError:
        continue

    # Find and add company name
    company = soup.select_one("p:contains(Name)").find_previous('p').text.strip()
    df['Company Name'] = company

    # 5. Adding Date
    date = soup.find('p', text = re.compile(r"(?:January|February|March|April|May|June|July|August|September|October|November|December)")).text.strip()
    df['Filing Date'] = date
    #print(df)

#Tidying and saving initial csv
temp_var = tables[0]
temp_var = pd.DataFrame(temp_var)
temp_var.columns = temp_var.iloc[0]
temp_var = temp_var.drop(temp_var.index[0])
temp_var = temp_var.drop(['nan'], axis = 1)
column_names = ['Beneficial_Owner', 'share','Shares voting power', 'percent','Percent shares', 'Company name', 'Date filed']
temp_var.columns = column_names
temp_var.to_csv("CSV FILE PATH HERE")
